In [1]:
import numpy as np
import pandas as pd
import datetime as dt

from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1339,tt0414387,28000000,121147947,Pride & Prejudice,Keira Knightley|Matthew Macfadyen|Rosamund Pik...,Joe Wright,A romance ahead of its time.,Pride and Prejudice is a humorous story of lov...,128,Drama|Romance,Universal Pictures|StudioCanal|Scion Films|Foc...,9/16/2005,7.6,2005
438,tt1075747,47000000,10539000,Jonah Hex,Josh Brolin|John Malkovich|Megan Fox|Michael F...,Jimmy Hayward,Revenge gets ugly.,Gunsliger Jonah Hex (Josh Brolin) is appointed...,80,Action|Western|Drama|Fantasy|Thriller,Warner Bros.,6/18/2010,4.4,2010
1611,tt0349683,120000000,203567857,King Arthur,Clive Owen|Ioan Gruffudd|Keira Knightley|Mads ...,Antoine Fuqua,Rule Your Fate.,"The story of the Arthurian legend, based on th...",126,Adventure|War|History,Jerry Bruckheimer Films|World 2000 Entertainme...,7/7/2004,6.1,2004
1647,tt0327437,110000000,72178895,Around the World in 80 Days,Jackie Chan|Steve Coogan|CÃ©cile De France|Rob...,Frank Coraci,Let your imagination soar.,"A bet pits a British inventor, a Chinese thief...",120,Action|Adventure|Comedy,Studio Babelsberg|Walden Media|Babelsberg Film...,6/16/2004,5.6,2004
1744,tt0416508,16500000,37311672,Becoming Jane,Anne Hathaway|James McAvoy|Julie Walters|James...,Julian Jarrold,Her own life is her greatest inspiration.,A biographical portrait of a pre-fame Jane Aus...,120,Drama|Romance,Ecosse Films|2 Entertain|Bueprint Pictures,3/2/2007,6.9,2007


# Предобработка

In [3]:
answers = {} # создадим словарь для ответов

In [4]:

#копия ДФ для предобработок
data_copy = data.copy()

#добавим прибыль
data_copy['profit'] = data_copy['revenue'] - data_copy['budget'] #столбец прибыли

#форматируем дату
data_copy['new_date'] = data_copy['release_date']
data_copy['new_date'] = pd.to_datetime(data_copy['new_date'], format = "%m/%d/%Y")

#разбиваем жанры, актеров, режиссеров, компании. Потом будем взрывать по необходимости
data_copy['genres'] = data_copy.genres.str.split('|')
data_copy['cast'] = data_copy.cast.str.split('|')
data_copy['director'] = data_copy.director.str.split('|')
data_copy['production_companies'] = data_copy.production_companies.str.split('|')
data_copy.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date
1027,tt1636826,12000000,100000000,Project X,"[Oliver Cooper, Jonathan Daniel Brown, Thomas ...",[Nima Nourizadeh],Witness it,Three high school seniors throw a party to mak...,88,"[Comedy, Crime]","[Silver Pictures, Green Hat Films]",3/1/2012,6.5,2012,88000000,2012-03-01
543,tt0209475,35000000,94728529,The Wedding Planner,"[Jennifer Lopez, Matthew McConaughey, Justin C...",[Adam Shankman],His big day is her big problem.,"Mary Fiore, San Francisco's premiere wedding p...",103,[Comedy],"[Columbia Pictures Corporation, Intermedia Fil...",1/26/2001,5.4,2001,59728529,2001-01-26
176,tt2626350,45000000,86165646,Step Up All In,"[Ryan Guzman, Briana Evigan, Stephen Boss, Mis...",[Trish Sie],Every Step Has Led To This,All-stars from the previous Step Up installmen...,112,"[Romance, Drama, Music]","[Summit Entertainment, Offspring Entertainment]",7/16/2014,6.7,2014,41165646,2014-07-16
381,tt0435761,200000000,1063171911,Toy Story 3,"[Tom Hanks, Tim Allen, Ned Beatty, Joan Cusack...",[Lee Unkrich],No toy gets left behind.,"Woody, Buzz, and the rest of Andy's toys haven...",103,"[Animation, Family, Comedy]","[Walt Disney Pictures, Pixar Animation Studios]",6/16/2010,7.5,2010,863171911,2010-06-16
940,tt0257360,30000000,105834556,About Schmidt,"[Jack Nicholson, Kathy Bates, Hope Davis, Derm...",[Alexander Payne],Schmidt Happens,66-year-old Warren Schmidt is a retired insura...,125,"[Drama, Comedy]","[New Line Cinema, Avery Pix]",12/13/2002,6.7,2002,75834556,2002-12-13


In [5]:
data_copy.describe()

,budget,revenue,runtime,vote_average,release_year,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,2.544506e+09


# 1. У какого фильма из списка самый большой бюджет?

In [6]:
data_copy.loc[data_copy.budget == data_copy.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...",[Rob Marshall],Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,"[Adventure, Action, Fantasy]","[Walt Disney Pictures, Jerry Bruckheimer Films...",5/11/2011,6.3,2011,641683000,2011-05-11


In [7]:
answers[1] = data_copy.loc[data_copy.budget == data_copy.budget.max(), 'original_title'].values[0]

In [8]:
answers[1]

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
data_copy.loc[data_copy.runtime == data_copy.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date
1157,tt0279111,56000000,12923936,Gods and Generals,"[Stephen Lang, Jeff Daniels, Robert Duvall, Ke...",[Ronald F. Maxwell],The nations heart was touched by...,The film centers mostly around the personal an...,214,"[Drama, History, War]","[Turner Pictures, Antietam Filmworks]",2/21/2003,5.8,2003,-43076064,2003-02-21


In [10]:
answers[2] = data_copy.loc[data_copy.runtime == data_copy.runtime.max(), 'original_title'].values[0]

In [11]:
answers[2]

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [12]:
data_copy.loc[data_copy.runtime == data_copy.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date
768,tt1449283,30000000,14460000,Winnie the Pooh,"[Jim Cummings, Travis Oates, Jim Cummings, Bud...","[Stephen Anderson, Don Hall]",Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,"[Animation, Family]","[Walt Disney Pictures, Walt Disney Animation S...",4/13/2011,6.8,2011,-15540000,2011-04-13


In [13]:
data_copy.loc[data_copy.runtime == data_copy.runtime.min(), 'original_title'].values[0]

'Winnie the Pooh'

In [14]:
answers[3] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [15]:
answers[4] = round(data_copy.runtime.mean())
answers[4]

110

# 5. Каково медианное значение длительности фильмов? 

In [16]:
answers[5] = round(data_copy.runtime.median())
answers[5]

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [17]:
answers[6] = data_copy.loc[data_copy.profit == data_copy.profit.max(), 'original_title'].values[0]
answers[6]

'Avatar'

# 7. Какой фильм самый убыточный? 

In [18]:
answers[7] = data_copy.loc[data_copy.profit == data_copy.profit.min(), 'original_title'].values[0]
answers[7]

'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [19]:
answers[8] = len(data.loc[data_copy.profit > 0])
answers[8]

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [20]:
question_9 = data_copy.loc[(data_copy.release_year==2008)].sort_values('revenue', ascending=False)
question_9.loc[question_9.revenue == question_9.revenue.max(), ['original_title']]

,original_title
599,The Dark Knight


In [21]:
answers[9] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [22]:
question_10 = data_copy.copy()
question_10.loc[question_10.profit == question_10.profit.min(), ['original_title', 'release_year']]

,original_title,release_year
1245,The Lone Ranger,2013


In [23]:
answers[10] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [24]:
genre_duplicate = data_copy.explode('genres')
genre_duplicate.genres.describe()

count      5107
unique       19
top       Drama
freq        782
Name: genres, dtype: object

In [25]:
answers[11] = 'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [26]:
genre_duplicate.loc[genre_duplicate.profit>0].genres.describe()

count      3962
unique       18
top       Drama
freq        560
Name: genres, dtype: object

In [27]:
answers[12] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [28]:
question_13 = data_copy.copy()
question_13.explode('director').groupby(['director']).sum().sort_values('profit', ascending=False)

,budget,revenue,runtime,vote_average,release_year,profit
director,,,,,,
Peter Jackson,1288000000,6490593685,1355,58.1,16059,5202593685
David Yates,775000000,4154295625,567,29.6,8037,3379295625
Christopher Nolan,1005000000,4167548502,1135,61.1,16057,3162548502
J.J. Abrams,740000000,3579169916,633,35.1,10054,2839169916
Michael Bay,1126000000,3886938960,1209,50.0,16063,2760938960
...,...,...,...,...,...,...
Jack Sholder,90000000,14828081,91,4.7,2000,-75171919
Peter Hyams,120000000,33043455,214,9.9,4006,-86956545
Ron Underwood,100000000,7103973,95,4.6,2002,-92896027


In [29]:
answers[13] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [30]:
question_14 = genre_duplicate.copy()
question_14 = question_14.explode('director')
question_14[question_14.genres == 'Action'].groupby(['director']).count().sort_values('genres', ascending=False)

,imdb_id,budget,revenue,original_title,cast,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date
director,,,,,,,,,,,,,,,
Robert Rodriguez,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
Michael Bay,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
Paul W.S. Anderson,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
Ridley Scott,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
Antoine Fuqua,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
John Whitesell,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Brian A Miller,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Jon Amiel,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [31]:
answers[14] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [32]:
actors_duplicate = data_copy.copy()
actors_duplicate = actors_duplicate.explode('cast')

actors_duplicate[actors_duplicate.release_year.astype(str) == '2012'].groupby('cast').sum().sort_values('revenue', ascending = False).head(10)

,budget,revenue,runtime,vote_average,release_year,profit
cast,,,,,,
Chris Hemsworth,485000000,2027450773,458,24.8,8048,1542450773
Denis Leary,310000000,1629460639,224,12.6,4024,1319460639
Anne Hathaway,311000000,1522851057,322,14.5,4024,1211851057
Robert Downey Jr.,220000000,1519557910,143,7.3,2012,1299557910
Mark Ruffalo,220000000,1519557910,143,7.3,2012,1299557910
Chris Evans,220000000,1519557910,143,7.3,2012,1299557910
Scarlett Johansson,220000000,1519557910,143,7.3,2012,1299557910
Ralph Fiennes,350000000,1409561013,242,12.3,4024,1059561013
Sacha Baron Cohen,271000000,1368110577,333,19.3,6036,1097110577


In [33]:
answers[15] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [34]:
actors_duplicate[actors_duplicate.budget > actors_duplicate.budget.mean()].groupby('cast').count().sort_values('revenue', ascending = False).head(3)

,imdb_id,budget,revenue,original_title,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date
cast,,,,,,,,,,,,,,,
Matt Damon,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18
Adam Sandler,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
Angelina Jolie,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16


In [35]:
answers[16] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [36]:
question_17 = data_copy.explode('cast')
question_17 = question_17.explode('genres')
question_17.loc[question_17.cast == 'Nicolas Cage'].groupby('genres').count().sort_values('revenue', ascending = False).head(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,production_companies,release_date,vote_average,release_year,profit,new_date
genres,,,,,,,,,,,,,,,
Action,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
Thriller,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15
Drama,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12


In [37]:
answers[17] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [38]:
question_18 = data_copy.explode('production_companies')
question_18[question_18.production_companies == 'Paramount Pictures'].sort_values('profit').head(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date
925,tt0267626,100000000,35168966,K-19: The Widowmaker,"[Harrison Ford, Liam Neeson, Peter Sarsgaard, ...",[Kathryn Bigelow],Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,"[Thriller, Drama, History]",Paramount Pictures,7/19/2002,6.0,2002,-64831034,2002-07-19
1123,tt0300556,80000000,19480739,Timeline,"[Paul Walker, Frances O'Connor, Gerard Butler,...",[Richard Donner],They had to travel into the past to save the f...,A group of archaeological students become trap...,116,"[Action, Adventure, Science Fiction]",Paramount Pictures,11/26/2003,5.1,2003,-60519261,2003-11-26
1722,tt0435705,70000000,18211013,Next,"[Nicolas Cage, Julianne Moore, Jessica Biel, T...",[Lee Tamahori],"If you can see the future, you can save it.",Las Vegas showroom magician Cris Johnson has a...,96,"[Action, Science Fiction, Thriller]",Paramount Pictures,4/24/2007,5.9,2007,-51788987,2007-04-24


In [39]:
answers[18] = 'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [40]:
data_copy.groupby(['release_year']).sum().sort_values('revenue', ascending = False).head(5)

,budget,revenue,runtime,vote_average,profit
release_year,,,,,
2015,6780630004,25449202382,13355,755.2,18668572378
2014,7008050000,23405862953,13444,774.5,16397812953
2013,7970620000,23213799791,15039,846.9,15243179791
2012,7002000000,23079001687,12145,682.1,16077001687
2011,7946550531,22676791872,15355,868.5,14730241341


In [41]:
answers[19] = '2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [42]:
question_20 = data_copy.explode('production_companies')
question_20 = question_20[question_20.production_companies.str.contains('Warner Bros.')]
question_20.groupby(['release_year']).sum().sort_values('profit', ascending = False).head(3)

,budget,revenue,runtime,vote_average,profit
release_year,,,,,
2014,953600000,3246549646,1683,95.2,2292949646
2007,982000000,3183675217,1721,95.4,2201675217
2008,916000000,3050595031,1366,72.0,2134595031


In [43]:
answers[20] = '2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [44]:
question_21 = data_copy.copy()
question_21['month'] = question_21['new_date'].dt.month
question_21.groupby('month').count()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date
month,,,,,,,,,,,,,,,,
1,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110
2,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135
3,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156
4,149,149,149,149,149,149,149,149,149,149,149,149,149,149,149,149
5,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140
6,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147
7,142,142,142,142,142,142,142,142,142,142,142,142,142,142,142,142
8,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161
9,227,227,227,227,227,227,227,227,227,227,227,227,227,227,227,227


In [45]:
answers[21] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [46]:
question_22 = data.copy()
question_22['release_date'] = pd.to_datetime(question_22['release_date'])
question_22.release_date = question_22.release_date.apply(lambda x: x.strftime('%B'))
question_22.query('release_date in ["June", "July", "August"]')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,June,6.5,2015
6,tt1340138,155000000,440603537,Terminator Genisys,Arnold Schwarzenegger|Jason Clarke|Emilia Clar...,Alan Taylor,Reset the future,"The year is 2029. John Connor, leader of the r...",125,Science Fiction|Action|Thriller|Adventure,Paramount Pictures|Skydance Productions,June,5.8,2015
8,tt2293640,74000000,1156730962,Minions,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda|Pierre Coffin,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",91,Family|Animation|Adventure|Comedy,Universal Pictures|Illumination Entertainment,June,6.5,2015
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,June,8.0,2015
13,tt2120120,88000000,243637091,Pixels,Adam Sandler|Michelle Monaghan|Peter Dinklage|...,Chris Columbus,Game On.,Video game experts are recruited by the milita...,105,Action|Comedy|Science Fiction,Columbia Pictures|Happy Madison Productions,July,5.8,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873,tt0160009,40000000,30199105,The Art of War,Wesley Snipes|Donald Sutherland|Maury Chaykin|...,Christian Duguay,Who is your foe?,When ruthless terrorists threaten to bring dow...,117,Action|Adventure|Thriller,Amen Ra Films|Franchise Pictures|Warner Bros.|...,August,5.7,2000
1875,tt0219854,65000000,69700000,The Kid,Bruce Willis|Spencer Breslin|Emily Mortimer|Li...,Jon Turteltaub,Nobody ever grows up quite like they imagined.,Powerful businessman Russ Duritz is self-absor...,104,Fantasy|Comedy|Family,Walt Disney Pictures,July,5.9,2000
1884,tt0120903,75000000,157299717,X-Men,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,Adventure|Action|Science Fiction,Twentieth Century Fox Film Corporation|Donners...,July,6.6,2000
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,Rene Russo|Jason Alexander|Piper Perabo|Randy ...,Des McAnuff,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,Adventure|Animation|Action|Comedy|Family,Universal Pictures|Capella International|KC Me...,June,4.0,2000


In [47]:
answers[22] = '450'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [48]:
question_23 = data.copy()
question_23['release_date'] = pd.to_datetime(question_23['release_date'])
question_23.release_date = question_23.release_date.apply(lambda x: x.strftime('%B'))
question_23['director'] = question_23.director.str.split('|')
question_23=question_23.explode('director')
question_23.query('release_date in ["December", "January", "February"]').value_counts('director').head(5)

director
Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
Martin Scorsese      4
Adam Shankman        4
dtype: int64

In [49]:
answers[23] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [50]:
question_24 = data_copy.copy()

In [51]:
question_24 = question_24.explode('production_companies')

In [52]:
question_24['title_len'] = question_24['original_title'].apply(len)
question_24

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date,title_len
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Universal Studios,6/9/2015,6.5,2015,1363528810,2015-06-09,14
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Amblin Entertainment,6/9/2015,6.5,2015,1363528810,2015-06-09,14
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Legendary Pictures,6/9/2015,6.5,2015,1363528810,2015-06-09,14
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Fuji Television Network,6/9/2015,6.5,2015,1363528810,2015-06-09,14
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Dentsu,6/9/2015,6.5,2015,1363528810,2015-06-09,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,tt0162983,40000000,36037909,Hanging Up,"[Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...",[Diane Keaton],Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,"[Comedy, Drama]",Laurence Mark Productions,2/16/2000,5.2,2000,-3962091,2000-02-16,10
1887,tt0162983,40000000,36037909,Hanging Up,"[Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...",[Diane Keaton],Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,"[Comedy, Drama]",Columbia Pictures Corporation,2/16/2000,5.2,2000,-3962091,2000-02-16,10
1887,tt0162983,40000000,36037909,Hanging Up,"[Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...",[Diane Keaton],Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,"[Comedy, Drama]",Global Entertainment Productions GmbH & Compan...,2/16/2000,5.2,2000,-3962091,2000-02-16,10
1887,tt0162983,40000000,36037909,Hanging Up,"[Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...",[Diane Keaton],Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,"[Comedy, Drama]",Nora Ephron Productions,2/16/2000,5.2,2000,-3962091,2000-02-16,10


In [53]:
question_24.groupby('production_companies').title_len.mean().sort_values()

production_companies
Global Entertainment Group     2.0
Ixtlan Productions             2.0
XM2 Productions                2.0
Berlanti Productions           3.0
Everest Entertainment          3.0
                              ... 
Polsky Films                  46.0
Museum Canada Productions     46.0
Dos Corazones                 47.0
Jim Henson Company, The       59.0
Four By Two Productions       83.0
Name: title_len, Length: 1771, dtype: float64

In [54]:
answers[24] = 'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [55]:
question_25 = data_copy.copy()
question_25 = question_25.explode('production_companies')
question_25

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_date
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Universal Studios,6/9/2015,6.5,2015,1363528810,2015-06-09
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Amblin Entertainment,6/9/2015,6.5,2015,1363528810,2015-06-09
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Legendary Pictures,6/9/2015,6.5,2015,1363528810,2015-06-09
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Fuji Television Network,6/9/2015,6.5,2015,1363528810,2015-06-09
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]",Dentsu,6/9/2015,6.5,2015,1363528810,2015-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,tt0162983,40000000,36037909,Hanging Up,"[Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...",[Diane Keaton],Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,"[Comedy, Drama]",Laurence Mark Productions,2/16/2000,5.2,2000,-3962091,2000-02-16
1887,tt0162983,40000000,36037909,Hanging Up,"[Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...",[Diane Keaton],Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,"[Comedy, Drama]",Columbia Pictures Corporation,2/16/2000,5.2,2000,-3962091,2000-02-16
1887,tt0162983,40000000,36037909,Hanging Up,"[Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...",[Diane Keaton],Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,"[Comedy, Drama]",Global Entertainment Productions GmbH & Compan...,2/16/2000,5.2,2000,-3962091,2000-02-16
1887,tt0162983,40000000,36037909,Hanging Up,"[Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...",[Diane Keaton],Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,"[Comedy, Drama]",Nora Ephron Productions,2/16/2000,5.2,2000,-3962091,2000-02-16


In [56]:
question_25['overview_len'] = question_25['overview'].str.count(' ') + 1
question_25['overview_len']

0       26
0       26
0       26
0       26
0       26
        ..
1887    24
1887    24
1887    24
1887    24
1888    43
Name: overview_len, Length: 6078, dtype: int64

In [57]:
question_25.groupby('production_companies').overview_len.mean().sort_values()

production_companies
Motion Picture Corporation of America     11.0
Empire Pictures                           11.0
Henceforth                                13.0
Phantom Four                              13.0
London Boulevard                          13.0
                                         ...  
Brookwell-McNamara Entertainment         156.0
98 MPH Productions                       159.0
Heineken Branded Entertainment           161.0
Room 9 Entertainment                     161.0
Midnight Picture Show                    175.0
Name: overview_len, Length: 1771, dtype: float64

In [58]:
answers[25] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [59]:
list_26 = ['Inside Out', 'The Dark Knight', '12 Years a Slave', 
           'BloodRayne', 'The Adventures of Rocky & Bullwinkle', 
           'Batman Begins', 'The Lord of the Rings: The Return of the King', 'Upside Down',
           '300', ' Lucky Number Slevin', 'Kill Bill: Vol. 1', 
           'Upside Down', 'Inside Out', 'Iron Man']
question_26 = data.copy() 
question_26 = question_26.sort_values('vote_average', ascending = False).head(round(len(data.index)/100))

In [60]:
question_26[question_26['original_title'].isin(list_26)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,6/9/2015,8.0,2015
1191,tt2024544,20000000,187000000,12 Years a Slave,Chiwetel Ejiofor|Michael Fassbender|Lupita Nyo...,Steve McQueen,The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,Drama|History,Plan B Entertainment|Regency Enterprises|River...,10/18/2013,7.9,2013
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Peter Jackson,The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,Adventure|Fantasy|Action,WingNut Films|New Line Cinema,12/1/2003,7.9,2003


In [61]:
answers[26] = ['Inside Out', 'The Dark Knight', '12 Years a Slave']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [62]:
import itertools

In [63]:
list_27_1 = ['Johnny Depp', 'Helena Bonham Carter']
list_27_2 = ['Ben Stiller ', 'Owen Wilson']
list_27_3 = ['Vin Diesel', 'Paul Walker']
list_27_4 = ['Adam Sandler', 'Kevin James']
list_27_5 = ['Daniel Radcliffe', 'Rupert Grint']

In [64]:
list_27 = ['Johnny Depp', 'Helena Bonham Carter', 
           'Ben Stiller ', 'Owen Wilson',
           'Vin Diesel', 'Paul Walker',
           'Adam Sandler', 'Kevin James',
           'Daniel Radcliffe', 'Rupert Grint']

In [65]:
question_27 = data.copy()

In [66]:
#т.к. пары актеров заранее известны, можно перебрать вручную 
question_27_list = question_27[question_27.cast.str.contains('Daniel Radcliffe')]

In [67]:
question_27_list = question_27_list[question_27_list.cast.str.contains('Rupert Grint')]

In [68]:
question_27_list

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
242,tt0417741,250000000,933959197,Harry Potter and the Half-Blood Prince,Daniel Radcliffe|Rupert Grint|Emma Watson|Tom ...,David Yates,Dark Secrets Revealed,"As Harry begins his sixth year at Hogwarts, he...",153,Adventure|Fantasy|Family,Warner Bros.|Heyday Films,7/7/2009,7.3,2009
374,tt0926084,250000000,954305868,Harry Potter and the Deathly Hallows: Part 1,Daniel Radcliffe|Emma Watson|Rupert Grint|Ralp...,David Yates,One Wayâ€¦ One Fateâ€¦ One Hero.,"Harry, Ron and Hermione walk away from their l...",146,Adventure|Fantasy|Family,Warner Bros.|Heyday Films,10/17/2010,7.4,2010
497,tt0241527,125000000,976475550,Harry Potter and the Philosopher's Stone,Daniel Radcliffe|Rupert Grint|Emma Watson|John...,Chris Columbus,Let the Magic Begin.,Harry Potter has lived under the stairs at his...,152,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films,11/16/2001,7.2,2001
722,tt1201607,125000000,1327817822,Harry Potter and the Deathly Hallows: Part 2,Daniel Radcliffe|Rupert Grint|Emma Watson|Alan...,David Yates,It all ends here.,"Harry, Ron and Hermione continue their quest t...",130,Adventure|Family|Fantasy,Warner Bros.|Heyday Films|Moving Picture Compa...,7/7/2011,7.7,2011
864,tt0295297,100000000,876688482,Harry Potter and the Chamber of Secrets,Daniel Radcliffe|Rupert Grint|Emma Watson|Kenn...,Chris Columbus,Hogwarts is back in session.,"Ignoring threats to his life, Harry returns to...",161,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films|MIRACL...,11/13/2002,7.2,2002
1312,tt0330373,150000000,895921036,Harry Potter and the Goblet of Fire,Daniel Radcliffe|Rupert Grint|Emma Watson|Ralp...,Mike Newell,Dark And Difficult Times Lie Ahead.,"Harry starts his fourth year at Hogwarts, comp...",157,Adventure|Fantasy|Family,Patalex IV Productions Limited|Warner Bros.|He...,11/5/2005,7.3,2005
1560,tt0304141,130000000,789804554,Harry Potter and the Prisoner of Azkaban,Daniel Radcliffe|Rupert Grint|Emma Watson|Gary...,Alfonso CuarÃ³n,Something wicked this way comes.,"Harry, Ron and Hermione return to Hogwarts for...",141,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films|P of A...,5/31/2004,7.4,2004
1670,tt0373889,150000000,938212738,Harry Potter and the Order of the Phoenix,Daniel Radcliffe|Rupert Grint|Emma Watson|Mich...,David Yates,Evil Must Be Confronted.,Returning for his fifth year of study at Hogwa...,138,Adventure|Fantasy|Family|Mystery,Cool Music|Warner Bros.|Heyday Films|Harry Pot...,6/28/2007,7.2,2007


In [69]:
answers[27] = list_27_5

# Submission

In [70]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{1: 'Pirates of the Caribbean: On Stranger Tides',
 2: 'Gods and Generals',
 3: 'Winnie the Pooh (tt1449283)',
 4: 110,
 5: 107,
 6: 'Avatar',
 7: 'The Lone Ranger',
 8: 1478,
 9: 'The Dark Knight (tt0468569)',
 10: 'The Lone Ranger (tt1210819)',
 11: 'Drama',
 12: 'Drama',
 13: 'Peter Jackson',
 14: 'Robert Rodriguez',
 15: 'Chris Hemsworth',
 16: 'Matt Damon',
 17: 'Action',
 18: 'K-19: The Widowmaker',
 19: '2015',
 20: '2014',
 21: 'Сентябрь',
 22: '450',
 23: 'Peter Jackson',
 24: 'Four By Two Productions',
 25: 'Midnight Picture Show',
 26: ['Inside Out', 'The Dark Knight', '12 Years a Slave'],
 27: ['Daniel Radcliffe', 'Rupert Grint']}

In [71]:
# и убедиться что ни чего не пропустил)
len(answers)

27